In [1]:
import numpy as np
import random
import torch

from torch.utils.data import DataLoader
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam, AdamW
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.cluster import MiniBatchKMeans
from scipy.cluster.vq import vq, kmeans

from qqdm import qqdm, format_str
import pandas as pd
import pdb

In [2]:
train = np.load("data-bin/trainingset.npy", allow_pickle = True)
test = np.load("data-bin/testingset.npy", allow_pickle = True)

print(train.shape)
print(test.shape)

(140001, 64, 64, 3)
(19999, 64, 64, 3)


In [3]:
def same_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(2021)

In [4]:
class fcn_autoencoder(nn.Module):
    def __init__(self):
        super(fcn_autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(64 * 64 * 3, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Linear(64, 12),
            nn.ReLU(True),
            nn.Linear(12, 3),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Linear(128, 64 * 64 * 3),
            nn.Tanh(),
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [18]:
class conv_autoencoder(nn.Module):
    def __init__(self):
        super(conv_autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 128, 4, stride = 2, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, stride = 2, padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, 4, stride = 2, padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # nn.Conv2d(48, 96, 4, stride = 2, padding = 1),
            # nn.ReLU(),
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 4, stride = 2, padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, stride = 2, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 3, 4, stride = 2, padding = 1),
            # nn.ReLU(),
            # nn.ConvTranspose2d(12, 3, 4, stride = 2, padding = 1),
            nn.Tanh(),
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [6]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride = 2, padding = 1),
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride = 2, padding = 1),
            nn.ReLU(),
        )
        
        self.enc_out_1 = nn.Sequential(
            nn.Conv2d(24, 48, 4, stride = 2, padding = 1),
            nn.ReLU(),
        )
        
        self.enc_out_2 = nn.Sequential(
            nn.Conv2d(24, 48, 4, stride = 2, padding = 1),
            nn.ReLU(),
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(48, 24, 4, stride = 2, padding = 1),
            nn.ReLU(),
            nn.ConvTranspose2d(24, 12, 4, stride = 2, padding = 1),
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride = 2, padding = 1),
            nn.Tanh(),
        )
    
    def encode(self, x):
        h1 = self.encoder(x)
        return self.enc_out_1(h1), self.enc_out_2(h1)
    
    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)
    
    def decode(self, x):
        return self.decoder(x)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

def loss_vae(recon_x, x, mu, logvar, criterion):
    mse = criterion(recon_x, x)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    return KLD + mse

In [7]:
class Resnet(nn.Module):
    def __init__(self, fc_hidden1 = 1024, fc_hidden2 = 768, drop_p = 0.3, CNN_embed_dim = 256):
        super(Resnet, self).__init__()
        
        self.fc_hidden1, self.fc_hidden2, self.CNN_embed_dim = fc_hidden1, fc_hidden2, CNN_embed_dim
        
        self.ch1, self.ch2, self.ch3, self.ch4 = 16, 32, 64, 128
        self.k1, self.k2, self.k3, self.k4 = (5, 5), (3, 3), (3, 3), (3, 3)
        self.s1, self.s2, self.s3, self.s4 = (2, 2), (2, 2), (2, 2), (2, 2)
        self.pd1, self.pd2, self.pd3, self.pd4 = (0, 0), (0, 0), (0, 0), (0, 0)
        
        resnet = models.resnet18(pretrained = False)
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        self.fc1 = nn.Linear(resnet.fc.in_features, self.fc_hidden1)
        self.bn1 = nn.BatchNorm1d(self.fc_hidden1, momentum = 0.01)
        self.fc2 = nn.Linear(self.fc_hidden1, self.fc_hidden2)
        self.bn2 = nn.BatchNorm1d(self.fc_hidden2, momentum = 0.01)
        
        self.fc3_mu = nn.Linear(self.fc_hidden2, self.CNN_embed_dim)
        
        self.fc4 = nn.Linear(self.CNN_embed_dim, self.fc_hidden2)
        self.bn4 = nn.BatchNorm1d(self.fc_hidden2)
        self.fc5 = nn.Linear(self.fc_hidden2, 64 * 4 * 4)
        self.bn5 = nn.BatchNorm1d(64 * 4 * 4)
        self.relu = nn.ReLU(inplace = True)
        
        self.convTrans6 = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size = self.k4, stride = self.s4, padding = self.pd4),
            nn.BatchNorm2d(32, momentum = 0.01),
            nn.ReLU(inplace = True),
        )
        self.convTrans7 = nn.Sequential(
            nn.ConvTranspose2d(32, 8, kernel_size = self.k3, stride = self.s3, padding = self.pd3),
            nn.BatchNorm2d(8, momentum = 0.01),
            nn.ReLU(inplace = True),
        )
        self.convTrans8 = nn.Sequential(
            nn.ConvTranspose2d(8, 3, kernel_size = self.k2, stride = self.s2, padding = self.pd2),
            nn.BatchNorm2d(3, momentum = 0.01),
            nn.Sigmoid(),
        )
        
    def encode(self, x):
        x = self.resnet(x)
        x = x.view(x.size(0), -1)
        
        if x.shape[0] > 1:
            x = self.bn1(self.fc1(x))
        else:
            x = self.fc1(x)
        x = self.relu(x)
        
        if x.shape[0] > 1:
            x = self.bn2(self.fc2(x))
        else:
            x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3_mu(x)
        return x
    
    def decode(self, z):
        if z.shape[0] > 1:
            x = self.relu(self.bn4(self.fc4(z)))
            x = self.relu(self.bn5(self.fc5(x)))
        else:
            x = self.relu(self.fc4(z))
            x = self.relu(self.fc5(x))
        
        x = x.view(-1, 64, 4, 4)
        x = self.convTrans6(x)
        x = self.convTrans7(x)
        x = self.convTrans8(x)
        x = F.interpolate(x, size = (64, 64), mode = 'bilinear', align_corners = True)
        return x
    
    def forward(self, x):
        x = self.encode(x)
        x_reconst = self.decode(x)
        return x_reconst

In [8]:
class CustomTensorDataset(TensorDataset):
    def __init__(self, tensors):
        self.tensors = tensors
        if tensors.shape[-1] == 3:
            self.tensors = tensors.permute(0, 3, 1, 2)
        self.transforms = transforms.Compose([
            transforms.Lambda(lambda x : x.to(torch.float32)),
            transforms.Lambda(lambda x : 2. * x / 255. - 1.),
        ])
        
    def __len__(self):
        return len(self.tensors)
    
    def __getitem__(self, index):
        tensor = self.tensors[index]
        if self.transforms:
            tensor = self.transforms(tensor)
        return tensor

In [19]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

tensors = torch.from_numpy(train)
train_data = CustomTensorDataset(tensors)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, sampler = train_sampler, batch_size = batch_size)

model_type = 'cnn'
model_classes = {'fcn' : fcn_autoencoder(), 'cnn' : conv_autoencoder(), 'vae' : VAE(), 'resnet' : Resnet()}
model = model_classes[model_type].cuda()

optimizer = Adam(model.parameters(), lr = learning_rate)
criterion = nn.MSELoss()

In [20]:
best_loss = np.inf
model.train()

qqdm_train = qqdm(range(num_epochs), desc = format_str('bold', 'Description'))

for epoch in qqdm_train:
    tot_loss = list()
    for data in train_loader:
        if model_type in ['cnn', 'vae', 'resnet']:
            imgs = data.float().cuda()
        else:
            imgs = data.float().cuda()
            imgs = imgs.view(imgs.shape[0], -1)
        
        output = model(imgs)
        
        if model_type in ['vae']:
            loss = loss_vae(output[0], imgs, output[1], output[2], criterion)
        else:
            loss = criterion(output, imgs)
            
        tot_loss.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    mean_loss = np.mean(tot_loss)
    qqdm_train.set_infos({
        'epoch': f'{epoch + 1:.0f}/{num_epochs:.0f}',
        'loss': f'{mean_loss:.4f}',
    })
    
    if mean_loss < best_loss:
        best_loss = mean_loss
        torch.save(model, 'best_model_{}.pt'.format(model_type))
    
    torch.save(model, 'last_model_{}.pt'.format(model_type))

 Iters    Elapsed Time      Speed                                                                                       
 0/100          -             -                                                                                         
Description   0.0% |                                                                                                   | Iters    Elapsed Time      Speed    epoch   loss                                                                       
 1/100  00:02:21<03:53:21  0.01it/s  1/100  0.0194                                                                      
Description   1.0% |                                                                                                   | Iters    Elapsed Time      Speed    epoch   loss                                                                       
 2/100  00:04:43<03:51:38  0.01it/s  2/100  0.0054                                                                      
Description   2.0% |█             

In [21]:
eval_batch_size = 200

data = torch.tensor(test, dtype = torch.float32)
test_data = CustomTensorDataset(data)
test_sampler = SequentialSampler(test_data)
test_loader = DataLoader(test_data, sampler = test_sampler, batch_size = eval_batch_size)

model_path = 'last_model_cnn.pt'
model = torch.load(model_path).cuda()
model.eval()

eval_criterion = nn.MSELoss(reduction = 'none')

out_file = 'PREDICTION_FILE.csv'

In [22]:
anomality = list()

with torch.no_grad():
    for i, data in enumerate(test_loader):
        if model_type in ['vae', 'cnn', 'resnet']:
            imgs = data.float().cuda()
        elif model_type in ['fcn']:
            imgs = data.float().cuda()
            imgs = imgs.view(imgs.shape[0], -1)
        else:
            imgs = data[0].cuda()
        
        output = model(imgs)
        if model_type in ['cnn', 'resnet', 'fcn']:
            output = output
        elif model_type in ['resvae']:
            output = output[0]
        elif model_type in ['vae']:
            output = output[0]
        
        if model_type in ['fcn']:
            eval_loss = eval_criterion(output, imgs).sum(-1)
        else:
            eval_loss = eval_criterion(output, imgs).sum([1, 2, 3])
        
        anomality.append(eval_loss)
    
    anomality = torch.cat(anomality, axis = 0)
    anomality = torch.sqrt(anomality).reshape(len(anomality), 1).cpu().numpy()
    
df = pd.DataFrame(anomality, columns = ['Predicted'])
df.to_csv(out_file, index_label = 'Id')